In [1]:
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from trusteval.dimension.robustness.robustness_llm import pipeline
from trusteval.dimension.robustness.robustness_llm.dataset import download_glue_data
from trusteval.dimension.robustness.robustness_llm.dataset import download_other_data
base_dir = "./test/robustness/"
dataset_dir = os.path.join(base_dir, "data")  

os.makedirs(base_dir, exist_ok=True)
os.makedirs(dataset_dir, exist_ok=True)  

args = [
    "--data_dir", dataset_dir,  
    "--tasks", "all" 
]

e:\software\AnaConda\envs\api\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d:\paper\TrustEval-toolkit\trusteval\dimension\robustness\robustness_llm\robustness_code\..\repo_tool\emoji-embeddings.jsonl.gz


In [2]:
download_glue_data.get_datasets(args)
datasets = download_other_data.get_datasets(cache_dir=dataset_dir)  

	Completed!
	Completed!
Processing MRPC...
	Error downloading standard development IDs for MRPC. You will need to manually split your data.
	Completed!
	Completed!
	Note (12/10/20): This script no longer downloads SNLI. You will need to manually download and format the data to use SNLI.
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


Generating validation split: 100%|██████████| 4887/4887 [00:00<00:00, 163863.84 examples/s]


2024-12-27 17:19:10,701 [ERROR] Failed to load cnn_dailymail dataset: At least one data file must be specified, but got data_files=None


In [3]:
await pipeline.pipeline(base_dir=base_dir)

  0%|          | 0/13 [00:00<?, ?it/s]





































































































































































































  8%|▊         | 1/13 [00:04<00:57,  4.79s/it]













Lazy loading embedding info ...


In [ ]:
from trusteval.src.response_generator.lm_response import generate_responses
import shutil
source_config = "../trusteval/dimension/robustness/robustness_llm/file_config.json"
target_config = os.path.join(base_dir, "file_config.json")

if os.path.exists(source_config):
    shutil.copy2(source_config, target_config)
    print(f"Successfully copied file_config to {target_config}")
else:
    print("Warning: Source file_config not found")

async_list = ['o1-preview','o1-mini','qwq-32B']

async def process_files_enhanced():
    await generate_responses(
        data_folder=base_dir,
        request_type="llm",
        async_list=async_list,
        prompt_key="enhanced_prompt",
        result_key="responses"
    )
    print("All files processed.")
await process_files_enhanced()

async def process_files_original():
    await generate_responses(
        data_folder=base_dir,
        request_type="llm",
        async_list=async_list,
        prompt_key="original_prompt",
        result_key="original_responses"
    )
    print("All files processed.")
await process_files_original()

In [2]:
from trusteval.src.evaluation import judge_responses
import shutil
config_path = os.path.join(parent_dir, 'trusteval/src/config/judge_prompt.yaml')
async_judge_model = ['gpt-4o-mini']
judge_key = 'judge'
judge_type='llm'
async_list = ['o1-preview','o1-mini','qwq-32B']

async def process_ground_truth():
    #process original
    source_config = "../trusteval/dimension/robustness/robustness_llm/file_config_ground_truth_ori.json"
    target_config = os.path.join(base_dir, "file_config.json")
    if os.path.exists(source_config):
        shutil.copy2(source_config, target_config)
        print(f"Successfully copied file_config to {target_config}")
    else:
        print("Warning: Source file_config not found")

    await judge_responses(
        data_folder=base_dir,
        async_judge_model=async_judge_model,
        target_models=async_list,
        judge_type=judge_type,
        response_key=['original_responses'],
        judge_key= 'ori_judge'
    )

    #process adv
    source_config = "../trusteval/dimension/robustness/robustness_llm/file_config_ground_truth.json"
    target_config = os.path.join(base_dir, "file_config.json")
    if os.path.exists(source_config):
        shutil.copy2(source_config, target_config)
        print(f"Successfully copied file_config to {target_config}")
    else:
        print("Warning: Source file_config not found")

    await judge_responses(
        data_folder=base_dir,
        async_judge_model=async_judge_model,
        target_models=async_list,
        judge_type=judge_type,
        response_key=['responses'],
        judge_key= 'adv_judge'
    )
await process_ground_truth()

async def process_open_ended():
    source_config = "../trusteval/dimension/robustness/robustness_llm/file_config_open_ended.json"
    target_config = os.path.join(base_dir, "file_config.json")
    response_key = ['responses','original_responses']
    
    if os.path.exists(source_config):
        shutil.copy2(source_config, target_config)
        print(f"Successfully copied file_config to {target_config}")
    else:
        print("Warning: Source file_config not found")

    await judge_responses(
        data_folder=base_dir,
        async_judge_model=async_judge_model,
        target_models=async_list,
        judge_type=judge_type,
        response_key=response_key,
        reverse_choice=True
    )
await process_open_ended()


Successfully copied file_config to ./test/robustness/file_config.json
Processing file: ground_truth_dataset.json with task type: with_groundtruth
File ./test/robustness/ground_truth_dataset_responses_judge.json already exists. Loading existing responses.


100%|██████████| 312/312 [00:00<00:00, 38510.43it/s]

Results saved to ./test/robustness/ground_truth_dataset_responses_judge.json


Successfully copied file_config to ./test/robustness/file_config.json
Processing file: ground_truth_dataset.json with task type: with_groundtruth_original
File ./test/robustness/ground_truth_dataset_responses_judge.json already exists. Loading existing responses.


100%|██████████| 312/312 [00:04<00:00, 66.80it/s]


Results saved to ./test/robustness/ground_truth_dataset_responses_judge.json
Successfully copied file_config to ./test/robustness/file_config.json
Processing file: open_ended_dataset.json with task type: robustness_open_ended
File ./test/robustness/open_ended_dataset_responses_judge.json already exists. Loading existing responses.


100%|██████████| 520/520 [00:15<00:00, 33.82it/s]


Results saved to ./test/robustness/open_ended_dataset_responses_judge.json


In [3]:
from trusteval.src.evaluation import lm_metric

lm_metric(
    base_dir=base_dir,
    aspect="robustness_llm",
    model_list=async_list
)

Processing robustness llm metrics: 100%|██████████| 520/520 [00:00<00:00, 260111.88it/s]

Metrics successfully exported to ./test/robustness/robustness_llm_metrics.csv
